# Genetic Algorithm
Looking at [Giba's property](https://www.kaggle.com/titericz/the-property-by-giba) made me wonder how to come up with this ordering of the rows and columns, and I thought that might be a problem suitable for genetic algorithms - whether that is actually the case, or if there is a much faster closed-form solution to this problem (?), I do not know. I've opted for implementing the algorithm from scratch rather than using a library, since this was very much done for my own education. I'm sure everything can be done better, faster, more pythonic etc. 

Starting out on this notebook earlier today I knew nothing about genetic algorithms, except the overall concepts [from this tutorial](https://blog.sicara.com/getting-started-genetic-algorithms-python-tutorial-81ffa1dd72f9) - now I reckon I might go buy a book to actually learn about it more thoroughly. Any recommendations would be awesome :) .. any comments/improvements for the code below would also be very much appreciated.

In [1]:
import gc

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook

from IPython.display import clear_output, display
from sklearn.externals.joblib import Parallel, delayed

# Giba's Property
For the purpose of this notebook I'll only look at the training df, and only at the small subset presented by Giba. I imagine the algorithm should scale pretty well to the entire dataset though, albeit with minor modifications when including test data. Let's first get the subset presented by Giba

In [2]:
#Remove constant features
def remove_constant_cols(df, tolerance=10):
    for c in df.columns:
        if len(df[c].unique()) <= tolerance:
            df.drop(c, axis=1, inplace=True)

In [3]:
# Get the data
train_df = pd.read_csv('train.csv').set_index('ID')

# Get columns and rows in question
giba_cols = [
    "f190486d6","58e2e02e6","eeb9cd3aa","9fd594eec","6eef030c1","15ace8c9f",
    "fb0f5dbfe","58e056e12","20aa07010","024c577b9","d6bb78916",
    "b43a7cfd5","58232a6fb"
]
giba_rows = [
    '7862786dc', 'c95732596', '16a02e67a', 'ad960f947', '8adafbb52',
    'fd0c7cfc2', 'a36b78ff7', 'e42aae1b8', '0b132f2c6', '448efbb28',
    'ca98b17ca', '2e57ec99f', 'fef33cb02'
]

giba_df = train_df.loc[giba_rows, :]
remove_constant_cols(giba_df)
giba_df

,target,20aa07010,bd8f989f1,22ed6dba3,80b14398e,8c94b6675,9ca0eee11,0f8d7b98e,0a69cc2be,408d191b3,...,98082c8ef,b850c3e18,2d065b147,a396ceeb9,d7568383a,8d7bfb911,dcfcddf16,1834f29f5,ea5ed6ff7,9437d8b64
ID,,,,,,,,,,,,,,,,,,,,,
7862786dc,3513333.34,440000.00,3450000.00,1964666.66,2002166.66,803538.46,2002166.66,4142666.66,5921285.72,5921285.72,...,5921285.72,730000.0,1515000.0,800000.00,17500000.0,0.0,520000.00,1.200000e+06,825000.00,10600000
c95732596,160000.00,0.00,3007600.00,273000.00,4119200.00,4184200.00,2002166.66,23641666.66,3403333.34,5921285.72,...,5921285.72,3420000.0,3464400.0,179000.00,660000.0,300000.0,2836333.34,0.000000e+00,148666.66,3920000
16a02e67a,2352551.72,1600000.00,572500.00,210000.00,34000.00,1446333.34,4119200.00,3938000.00,4620000.00,3403333.34,...,5921285.72,2803500.0,3234000.0,120400.00,0.0,1256000.0,677666.66,1.240000e+05,1030000.00,0
ad960f947,280000.00,466461.54,20081000.00,168000.00,1213900.00,1901000.00,34000.00,102000.00,812666.66,4620000.00,...,3403333.34,0.0,730000.0,792750.00,36060000.0,1650000.0,400000.00,0.000000e+00,1515000.00,44320000
8adafbb52,5450500.00,3147200.00,7262000.00,800000.00,150000.00,890000.00,1213900.00,6728666.66,490000.00,812666.66,...,4620000.00,4069000.0,3420000.0,609000.00,2596000.0,0.0,99000.00,1.073333e+05,3464400.00,0
fd0c7cfc2,1359000.00,75000.00,746666.66,179000.00,0.00,5466666.66,150000.00,13000000.00,4458000.00,490000.00,...,812666.66,4345200.0,2803500.0,350000.00,3364000.0,176000.0,91142.86,5.150000e+05,3234000.00,2800000
a36b78ff7,60000.00,1586888.88,1978000.00,120400.00,1509600.00,844666.66,0.00,10000000.00,8455500.00,4458000.00,...,490000.00,250000.0,0.0,0.00,11728000.0,22800000.0,600000.00,1.300000e+08,730000.00,2310000
e42aae1b8,12000000.00,1477600.00,19620000.00,792750.00,4406363.64,4846000.00,1509600.00,0.00,704000.00,8455500.00,...,4458000.00,364000.0,4069000.0,0.00,7640000.0,8319500.0,111600.00,8.800000e+04,3420000.00,0
0b132f2c6,500000.00,0.00,1249600.00,609000.00,849000.00,9206333.34,4406363.64,5680000.00,0.00,704000.00,...,8455500.00,2918000.0,4345200.0,0.00,360000.0,3730000.0,0.00,1.093333e+05,2803500.00,11984000


# Ordering rows & columns with Genetic Algorithm
It's pretty easy to see the structure in the above - timeseries in columns and rows, and column `f190486d6` is two steps ahead of the target. The following is my quick-n-dirty class with fitness function, breeding functions, mutation functions, etc. 

One thing to note in the `fitness()` function is that I insert the `target` and `target+1` into the dataframe before score evaluation - I do this simply to direct it towards the structure above, but I reckon it isn't strictly neccesary. This should work for the entire training set as well, but for the test set one would have to modify it, especially if test&train rows are intermingled. For now I just look at Giba's subset.

In [ ]:
class GeneticOptimizer():
    
    def __init__(self, 
                 n_population=100,
                 n_cols=40,
                 n_rows=40,
                 n_breeders=10, 
                 n_lucky=2,
                 renew_ratio=0.3,
                 n_generations=10, 
                 max_row_mutations=10, 
                 max_col_mutations=10, 
                 max_combined_rows=10, 
                 max_combined_cols=10,
                 optimize_rows=False):
        
        # Set variables
        self.n_population = n_population
        self.n_cols = n_cols
        self.n_rows = n_rows
        self.n_generations = n_generations
        self.n_breeders = n_breeders
        self.n_lucky = n_lucky
        self.renew_ratio = renew_ratio
        self.max_row_mutations = max_row_mutations
        self.max_col_mutations = max_col_mutations
        self.max_combined_rows = max_combined_rows
        self.max_combined_cols = max_combined_cols
        self.optimize_rows = optimize_rows
        self.history = []
        self.fittest = []
    
    @staticmethod
    def fitness(X, weights, individual):
        """
        Lower score means better alignment, see sample df at:
        https://www.kaggle.com/titericz/the-property-by-giba
        """
        # Get a copy of our dataframe       
        X = X.loc[individual['rows'], ['target','target+1'] + individual['cols'].tolist()]
        
        # Shift matrix to get fitness
        shiftLeftUp = X.iloc[1:, 1:].values
        deleteRightDown = X.iloc[:-1, :-1].values    

        # Calculate & return score
        diff = (shiftLeftUp - deleteRightDown).astype(bool)
        both_zero = np.logical_or(~shiftLeftUp.astype(bool),
                                  ~deleteRightDown.astype(bool))
        # Penalize score by number of zeroes in columns
        score = np.sum(np.logical_or(both_zero, diff) * weights)
        return score
    
    @staticmethod
    def hash_individual(individual):
        return hash(frozenset(individual))
    
    @staticmethod
    def swap_random(seq, n):
        """Swaps a n-length subsequence around in seq"""
        l = len(seq)
        idx = range(l)
        i1, i2 = np.random.choice(idx, 2, replace=False)
        i1 = l-n if n + i1 >= l else i1
        i2 = l-n if n + i2 >= l else i2
        for m in range(n):
            seq[i1+m], seq[i2+m] = seq[i2+m], seq[i1+m]
            
    @staticmethod
    def get_parallel(verbose=0, n_jobs=-1, pre_dispatch='2*n_jobs'):
        return Parallel(
            n_jobs=n_jobs,
            pre_dispatch=pre_dispatch,
            verbose=verbose
        )
            
    def create_random_population(self, n_pop, columns, index):
        population = []
        for _ in range(n_pop):
            np.random.shuffle(columns)
            if self.optimize_rows:
                np.random.shuffle(index)
            population.append({'cols': np.copy(columns)[:self.n_cols],
                               'rows': np.copy(index)[:self.n_rows]})
        return np.array(population)
    
    def compute_population_performance(self, population, X, weights, **kwargs):        
        parallel = self.get_parallel(**kwargs)
        performance = parallel(
            delayed(self.fitness)(X, weights, individual) for individual in population
        )
        return np.array(performance)
    
    def select_from_population(self, population, performance, best_sample=3, lucky_few=1):
        
        # Sort the population to have best first
        sorted_population = population[np.argsort(performance)]
        
        # Save the fittest individual of the generation
        self.fittest.append(sorted_population[0])
        
        # Create next generation with best and random
        nextGeneration = []
        for i in range(best_sample):
            nextGeneration.append(sorted_population[i])
        for i in range(lucky_few):
            nextGeneration.append(np.random.choice(sorted_population))
            
        # Shuffle new generation and return
        np.random.shuffle(nextGeneration)        
        return nextGeneration
    
    def create_child(self, breeders):
        
        # Mom, dad and child
        mom = breeders[np.random.randint(0, len(breeders))]
        dad = breeders[np.random.randint(0, len(breeders))]        
        child_columns, child_index = [0]*self.n_cols, [0]*self.n_rows
        
        # Convenience function
        def set_trait(array, index, mom_trait, dad_trait):
            if np.random.rand() > 0.5:
                if mom_trait not in array:
                    array[index] = mom_trait
            else:
                if dad_trait not in array:
                    array[index] = dad_trait
        
        # Get characteristics from parent 1
        for i in range(self.n_cols):
            set_trait(child_columns, i, mom['cols'][i], dad['cols'][i])
        if self.optimize_rows:
            for i in range(self.n_rows):
                set_trait(child_index, i, mom['rows'][i], dad['rows'][i])
            
        # Fill in missing values (in a sense also a mutation factor)
        missing_cols = [c for c in mom['cols'] if c not in child_columns]
        for i in range(self.n_cols):
            if child_columns[i] == 0:
                child_columns[i] = missing_cols.pop()
        
        if self.optimize_rows:
            missing_rows = [c for c in mom['rows'] if c not in child_index]
            for i in range(self.n_rows):
                if child_index[i] == 0:
                    child_index[i] = missing_rows.pop()
        else:
            child_index = mom['rows']
                
        return {'cols': np.array(child_columns), 'rows': np.array(child_index)}
    
    def create_children(self, breeders, n_children, **kwargs):
        parallel = self.get_parallel(**kwargs)
        nextPopulation = parallel(
            delayed(self.create_child)(breeders) for _ in range(n_children))
        return np.array(nextPopulation)
    
    def mutate_individual(self, individual):
        if self.optimize_rows and self.max_row_mutations > 0:
            for _ in np.arange(0, np.random.randint(0, self.max_row_mutations)):
                n = np.random.randint(1, self.max_combined_rows)
                self.swap_random(individual['rows'], n)
        if self.max_col_mutations > 0:
            for _ in np.arange(0, np.random.randint(0, self.max_col_mutations)):
                n = np.random.randint(1, self.max_combined_cols)
                self.swap_random(individual['cols'], n)
        return individual
    
    def mutate_population(self, population, **kwargs):
        parallel = self.get_parallel(**kwargs)
        nextPopulation = parallel(
            delayed(self.mutate_individual)(individual) for individual in population
        )
        return np.array(nextPopulation)
    
    def get_fittest_target_error(self, X, validation_index):
        """Assume first column in individual is 2 steps behind target"""
        
        individual = self.fittest[-1]
        
        target_idx = [i for i in individual['rows'] if i in validation_index]
        target = np.log1p(X.loc[target_idx, 'target'])
        
        target2p_col = individual['cols'][0]
        target2p = np.log1p(X.loc[target_idx, target2p_col].shift(-2))
        
        return np.sqrt((target-target2p).dropna()**2).sum()        
    
    def fit(self, X, y, weights=None, validation_index=None, **kwargs): 
        
        # Do not modify original
        X = X.copy()     
        # Create initial population
        population = self.create_random_population(self.n_population,
                                                   X.columns.tolist(),
                                                   X.index.tolist())
        # Add target and target+1 to X, so as to direct the order of result
        X.insert(0, 'target+1', y.shift(1))
        X.insert(0, 'target', y)
        X.fillna(0, inplace=True)
        
        # If no weights specified, all columns equally important
        if weights is None:
            weights = np.ones(self.n_cols+1)
        
        # Run the algorithm for n_generations
        for epoch in range(self.n_generations):
            
            # Get performance for each individual in population            
            performance = self.compute_population_performance(population, X, weights, **kwargs)
            
            # Get breeders
            breeders = self.select_from_population(population, performance)
            
            # If we have a validation index, then get the train error for the best performer
            if validation_index is not None:
                train_error = self.get_fittest_target_error(X, validation_index)
            else:
                train_error = 'NaN'   
            
            # Update population
            new_pop = self.create_random_population(int(self.n_population * self.renew_ratio),
                                       X.columns.tolist(),
                                       X.index.tolist())
            offspring = self.create_children(breeders, self.n_population-len(new_pop), **kwargs)

            population = np.concatenate([new_pop, offspring], axis=-1)
            
            # Mutate population before next generation
            population = self.mutate_population(population, **kwargs)            
            
            # Save to history & display
            clear_output()
            self.history.append({
                "pop_loss": np.mean(performance),
                "std_pop_loss": np.std(performance),
                "top_performer_loss": np.min(performance),
                'generation': epoch+1,
                'Train RMSLE': train_error
            })
            display(pd.DataFrame(self.history).set_index('generation'))
            
            # Just in case
            gc.collect()

This class basically creates an initially fully random population of column/row orders, and based on this breeds new combinations which minimize the fitness function - the lower the fitness function score, the close we are to a matrix that has the structure observed in Giba's subset. Let's try to run it for a few generations.

In [5]:
x = giba_df.drop(['target'], axis=1).iloc[:, :40]
x[x == 0].count().sum()

64

In [ ]:
# Number of cols to find
n_cols = 10

# Weigh different columns differently in scoring (most important are those close to target)
weights = np.exp(-np.linspace(0, np.sqrt(n_cols+1), n_cols+1))

# Instantiate class and run on training data        
gp_opt = GeneticOptimizer(
    n_population=10000,
    n_cols=n_cols,
    n_rows=13,
    n_breeders=100,
    n_lucky=10,
    renew_ratio=0.3,
    n_generations=15,
    max_row_mutations=5,
    max_col_mutations=5,
    max_combined_rows=5,
    max_combined_cols=5,
    optimize_rows=False
)

# Fit to data
gp_opt.fit(
    giba_df.drop(['target'], axis=1), giba_df['target'], 
    n_jobs=8,
    verbose=1,
    weights=weights,
    validation_index=giba_df.index.values
)

,Train RMSLE,pop_loss,std_pop_loss,top_performer_loss
generation,,,,
1,0.000000,29.278039,0.725057,20.806368
2,0.000000,29.273356,0.755028,19.796412
3,0.000000,29.258746,0.801838,17.443544
4,0.000000,29.269756,0.751565,20.431312
5,0.000000,29.271978,0.763110,15.844060
6,0.000000,29.277581,0.743503,18.591540
7,0.000000,29.293047,0.674719,20.995417
8,0.000000,29.280151,0.739912,17.443544
9,0.000000,29.267806,0.768715,17.443544


[Parallel(n_jobs=8)]: Done 912 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 10000 out of 10000 | elapsed:    2.3s finished
[Parallel(n_jobs=8)]: Done 668 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 7000 out of 7000 | elapsed:    1.7s finished
[Parallel(n_jobs=8)]: Done 644 tasks      | elapsed:    0.6s


Locally I've managed to get a top performer that matched Giba's solution perfectly (more generations, and slightly different population settings). I imagine this approach will scale well to the entire training (and test, with modifications), where the best solution may be less neat.

In [15]:
best = gp_opt.fittest[-1]
giba_df.loc[best['rows'], ['target'] + best['cols'].tolist()]

,target,5030aed26,b850c3e18,75b663d7d,d6bb78916,7d287013b,be4729cb7,8c94b6675,b7c931383,44f3640e4,408d191b3
ID,,,,,,,,,,,
7862786dc,3513333.34,3420000.0,730000.0,580000.00,550000.00,300000.0,4184200.00,803538.46,158000.00,0.00,5921285.72
c95732596,160000.00,2803500.0,3420000.0,3010000.00,3076666.66,1256000.0,1446333.34,4184200.00,274000.00,660000.00,5921285.72
16a02e67a,2352551.72,0.0,2803500.0,19620000.00,440000.00,1650000.0,1901000.00,1446333.34,332000.00,4963000.00,3403333.34
ad960f947,280000.00,4069000.0,0.0,1970000.00,0.00,0.0,890000.00,1901000.00,194000.00,332000.00,4620000.00
8adafbb52,5450500.00,4345200.0,4069000.0,758000.00,1600000.00,176000.0,5466666.66,890000.00,377333.34,0.00,812666.66
fd0c7cfc2,1359000.00,250000.0,4345200.0,643000.00,466461.54,22800000.0,844666.66,5466666.66,260000.00,90000.00,490000.00
a36b78ff7,60000.00,364000.0,250000.0,3057333.34,3147200.00,8319500.0,4846000.00,844666.66,440000.00,400000.00,4458000.00
e42aae1b8,12000000.00,2918000.0,364000.0,4003750.00,75000.00,3730000.0,9206333.34,4846000.00,0.00,452750.00,8455500.00
0b132f2c6,500000.00,1950000.0,2918000.0,0.00,1586888.88,11998000.0,3335000.00,9206333.34,1800000.00,361714.28,704000.00


In [8]:
giba_cols = [
    "f190486d6","58e2e02e6","eeb9cd3aa","9fd594eec","6eef030c1","15ace8c9f",
    "fb0f5dbfe","58e056e12","20aa07010","024c577b9","d6bb78916",
    "b43a7cfd5","58232a6fb"
]
giba_rows = [
    '7862786dc', 'c95732596', '16a02e67a', 'ad960f947', '8adafbb52',
    'fd0c7cfc2', 'a36b78ff7', 'e42aae1b8', '0b132f2c6', '448efbb28',
    'ca98b17ca', '2e57ec99f', 'fef33cb02'
]